- 首先尝试 svm 的分析性能
- 可能再尝试 simCLR 对比学习架构（效果可能会很差）

# 数据

In [1]:
!pip install -q "labelbox[data]"

     |████████████████████████████████| 167 kB 4.5 MB/s 
     |████████████████████████████████| 11.1 MB 54.0 MB/s 
     |████████████████████████████████| 6.3 MB 36.5 MB/s 


In [2]:
from labelbox import Client, OntologyBuilder
from labelbox.data.annotation_types import Geometry
from labelbox.data.annotation_types.collection import LabelList
from PIL import Image
import numpy as np
import os
import torch

In [3]:
class SegClsName:
  VESSEL = "血管"
  BRONCHUS = "支气管"

  def get_all_names():
    return ['支气管', '血管']

class VesselRatingName:
  D = "D血管周围浸润"

class BronchusRatingName:
  A = "A支气管浸润"
  B = "B支气管浸润定性"
  C = "C支气管腔渗出"

  def get_all_names():
    return ["A支气管浸润", "B支气管浸润定性", "C支气管腔渗出"]

In [4]:
API_KEY = "YOURS"
PROJECT_ID = "YOURS"
client = Client(api_key=API_KEY)
project = client.get_project(PROJECT_ID)
labels = project.label_generator().as_list()

/usr/local/lib/python3.7/dist-packages/labelbox/data/annotation_types/classification/classification.py:85: UserWarning: Dropdown classification is deprecated and will be removed in a future release
  warnings.warn("Dropdown classification is deprecated and will be "


In [5]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
import cv2
from matplotlib import pyplot as plt

In [6]:
from collections import Counter
from threading import Thread

def get_box(a):
  mask = a.value.draw()[:, :, 0]
  pos = np.where(mask != 0)
  xmin = np.min(pos[1])
  xmax = np.max(pos[1])
  ymin = np.min(pos[0])
  ymax = np.max(pos[0])
  # 只针对 ResNet18 的约束
  if xmax - xmin > 32 or ymax - ymin > 32: 
    return [xmin, ymin, xmax, ymax] 
  return None

def data_enhance(bs, rs):
  data_dict = {}
  for b,r in zip(bs, rs):
    if r not in data_dict.keys(): data_dict[r] = []
    data_dict[r].append(b)
  ctr = Counter(rs)
  max_num = max(list(ctr.values()))
  for r in data_dict.keys():
    if ctr[r] >= max_num: continue
    bs.extend(np.random.choice(data_dict[r], size=max_num-ctr[r]))
    rs.extend([r] * (max_num - ctr[r]))
    assert len(bs)==len(rs)

def get_box_ratings(lb_labels: LabelList, cls_name, rating_name, n_workers=4, reshape_size=(200,200), balance=False, flatten=True):

  boxes = []
  ratings = []

  def parse(lb, a):
    if a.name != cls_name: return
    rating = [float(c.value.answer.name) for c in a.classifications if c.name == rating_name]
    if len(rating) == 0: return
    rating = rating[0]
    box = get_box(a)
    if box == None: return
    im = lb.data.value[box[1]:box[3]+1, box[0]:box[2]+1]
    
    boxes.append(im)
    ratings.append(rating)

  class ParsingThread(Thread):
    def __init__(self, lb, a):
      Thread.__init__(self)
      self.lb = lb
      self.a = a

    def run(self):
      parse(self.lb, self.a)

  ths = []
  for lb in lb_labels:
    for a in lb.object_annotations():
      ths.append(ParsingThread(lb, a))

  for t in ths[:n_workers]: t.start()
  for t1, t2 in zip(ths, ths[n_workers:]):
    t1.join()
    t2.start()
  for t in ths[-n_workers:]: t.join()

  if balance: data_enhance(boxes, ratings)

  assert reshape_size != None
  boxes = np.array([np.array(Image.fromarray(im).resize(reshape_size), copy=True) for im in boxes])
  if flatten: boxes = np.array([im.flatten() for im in boxes])

  return np.array(boxes), np.array(ratings)

In [22]:
X, Y = get_box_ratings(labels, SegClsName.BRONCHUS, BronchusRatingName.B, n_workers=8)

In [28]:

indices = np.arange(len(Y))
np.random.shuffle(indices)
test_X, test_Y = X[indices[-5:]], Y[indices[-5:]]
train_X, train_Y = X[indices[:-5]], Y[indices[:-5]]

train_X.shape, train_Y.shape

((103, 120000), (103,))

In [29]:
from sklearn.svm import SVR

model = SVR(C=1.0, epsilon=0.2)
model.fit(train_X, train_Y)
train_X.shape, train_Y.shape

((103, 120000), (103,))

In [30]:
for tx, ty in zip(test_X, test_Y):
  pred_y = model.predict(np.array([tx]))
  mse = sum((pred_y-np.array([ty]))**2)
  print(mse, np.sqrt(mse))
pred_y = model.predict(test_X)
mse = sum((pred_y-test_Y)**2)/len(test_Y)
print(mse, np.sqrt(mse))


0.43820927751889277 0.6619737740416102
0.0016122428243471427 0.04015274367147459
0.5081636528599923 0.7128559832532742
0.3440107402712221 0.5865242878783641
0.02158923563678935 0.14693275889599755
0.2627170298222487 0.5125592939575369


In [31]:
model.score(test_X, test_Y) 

0.2273028534639745

In [32]:
import torch.nn as nn
import torchvision.models as models



class ResNetSimCLR(nn.Module):

    def __init__(self, base_model, out_dim):
        super(ResNetSimCLR, self).__init__()
        self.resnet_dict = {"resnet18": models.resnet18(pretrained=False, num_classes=out_dim),
                            "resnet50": models.resnet50(pretrained=False, num_classes=out_dim)}

        self.backbone = self._get_basemodel(base_model)
        dim_mlp = self.backbone.fc.in_features

        # add mlp projection head
        self.backbone.fc = nn.Sequential(nn.Linear(dim_mlp, dim_mlp), nn.ReLU(), self.backbone.fc)

    def _get_basemodel(self, model_name):
        try:
            model = self.resnet_dict[model_name]
        except KeyError:
            raise NotImplementedError(
                "Invalid backbone architecture. Check the config file and pass one of: resnet18 or resnet50")
        else:
            return model

    def forward(self, x):
        return self.backbone(x)